In [30]:
import cobrakbase
import cobra
import copy
import json

In [10]:
token = "DGXHUU34PUCT2V5JGUYA23P6TMQJM3FK"
kbase = cobrakbase.KBaseAPI(token)

In [18]:
model_json = None
with open('/Users/fliu/workspace/jupyter/data/sbml/awful_models/6.json', 'r') as fh:
    model_json = json.loads(fh.read())

In [19]:
model_json.keys()

dict_keys(['__VERSION__', 'biomasses', 'delete_biomasses', 'deleted_reactions', 'gapfilledcandidates', 'gapfillings', 'gapgens', 'genome_ref', 'id', 'model_edits', 'modelcompartments', 'modelcompounds', 'modelreactions', 'name', 'quantopts', 'source', 'source_id', 'template_ref', 'template_refs', 'type'])

In [41]:
fva = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0.5, )

In [21]:
media = kbase.get_object('Carbon-D-Glucose', 'filipeliu:narrative_1582914694010')
model = cobrakbase.core.converters.KBaseFBAModelToCobraBuilder(cobrakbase.core.KBaseFBAModel(model_json)).with_media(cobrakbase.core.KBaseBiochemMedia(media)).build()
model

Name,FBA_test_model
Memory address,0x07ffe545dd890
Number of metabolites,1287
Number of reactions,1563
Number of groups,0
Objective expression,1.0*bio1_biomass - 1.0*bio1_biomass_reverse_6e711
Compartments,"c0, e0"


In [42]:
fva_min_max = {}
for row_id, d in fva.iterrows():
    fva_min_max[row_id] = (d['minimum'], d['maximum'])

In [45]:
for r in model.reactions:
    lb = r.lower_bound
    ub = r.upper_bound
    r.lower_bound = 0 
    r.upper_bound = 0
    solution = model.optimize()
    r.lower_bound = lb 
    r.upper_bound = ub
    print("{}\t{}\t{}\t{}\t{}\t{}".format(r.id, r.lower_bound , r.upper_bound, solution.objective_value, fva_min_max[r.id][0], fva_min_max[r.id][1]))
    break

rxn02201_c0	0	1000	0.6978327749368387	0.0	1000.0


In [100]:
from cobra.core import Reaction, Metabolite, Model
a = Metabolite('A', "A", 'A')
ap = Metabolite('Ap', "A'", 'A')
app = Metabolite('App', "A''", 'A')
appp = Metabolite('Appp', "A'''", 'A')
b = Metabolite('B', "B", 'B')
bp = Metabolite('Bp', "B'", 'B')
ab = Metabolite('AB', "AB", 'AB')
apb = Metabolite('ApB', "A'B", 'AB')
a_ap = Reaction('A_Ap', "A -> A'", "", 0, 10)
a_ap.add_metabolites({a : -1, ap : 1})
print(a_ap)
ap_app = Reaction('Ap_App', "A' <-> A''", "", -10, 10)
ap_app.add_metabolites({ap : -1, app : 1})
print(ap_app)
app_appp = Reaction('App_Appp', "A'' -> A'''", "", 0, 10)
app_appp.add_metabolites({app : -1, appp : 1})
print(app_appp)
ab_app = Reaction('AB_AppB', "AB -> A'' + B", "", 0, 10)
ab_app.add_metabolites({ab : -1, b : 1, app : 1})
print(ab_app)
app_ab = Reaction('AppB_AB', "A' + B -> A'B", "", 0, 10)
app_ab.add_metabolites({b : -1, ap : -1, apb : 1})
print(app_ab)
apb_appp = Reaction('ApB_Appp', "A'B -> A'' + B'", "", 0, 10)
apb_appp.add_metabolites({apb : -1, appp : 1, bp : 1})

IN_A = Reaction('IN_A', "-> A", "", -5, 0)
IN_A.add_metabolites({a : -1})
IN_AB = Reaction('IN_AB', "-> AB", "", -5, 0)
IN_AB.add_metabolites({ab : -1})
OUT_Appp = Reaction('OUT_Appp', "-> A'''", "", 0, 5)
OUT_Appp.add_metabolites({appp : -1})
OUT_Bp = Reaction('OUT_Bp', "-> A'''", "", 0, 5)
OUT_Bp.add_metabolites({bp : -1})

print('!')
model = Model()
model.add_reactions([a_ap, ap_app, app_appp, ab_app, app_ab, apb_appp, OUT_Bp, IN_AB, IN_A, OUT_Appp])
model.objective = 'OUT_Appp'
solution = model.optimize()
fva = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=1)
for r in model.reactions:
    print(r, solution.fluxes[r.id])

A_Ap: A --> Ap
Ap_App: Ap <=> App
App_Appp: App --> Appp
AB_AppB: AB --> App + B
AppB_AB: Ap + B --> ApB
!
A_Ap: A --> Ap 5.0
Ap_App: Ap <=> App 5.0
App_Appp: App --> Appp 5.0
AB_AppB: AB --> App + B 0.0
AppB_AB: Ap + B --> ApB 0.0
ApB_Appp: ApB --> Appp + Bp 0.0
OUT_Bp: Bp -->  -0.0
IN_AB: AB <--  0.0
IN_A: A <--  -5.0
OUT_Appp: Appp -->  5.0


In [101]:
fva

,minimum,maximum
A_Ap,0.0,5.0
Ap_App,-5.0,5.0
App_Appp,0.0,5.0
AB_AppB,0.0,5.0
AppB_AB,0.0,5.0
ApB_Appp,0.0,5.0
OUT_Bp,0.0,5.0
IN_AB,-5.0,0.0
IN_A,-5.0,0.0
OUT_Appp,5.0,5.0


In [88]:
fva

,minimum,maximum
A_Ap,5.0,5.0
Ap_App,5.0,5.0
App_Appp,5.0,5.0
IN_A,-5.0,-5.0
OUT_Appp,5.0,5.0


In [ ]:
model.summary

0.6978327749368377

In [3]:
# Active KBase Template
template_v2 = kbase.get_object('GramNegModelTemplateV2', 'NewKBaseModelTemplates')
# Updated KBase Template
template_v2z = kbase.get_object('template_v2.z_06102020', 'filipeliu:narrative_1582914694010')
# Updated Git version Template
#template_v3 = kbase.get_object('GramNegative', 'chenry:narrative_1591451745118')
# Updated Git version Template Clean
#template_v3x = kbase.get_object('template_v3.x_06102020', 'filipeliu:narrative_1582914694010')
# Updated Git version Template Clean
#template_v3z = kbase.get_object('template_v3.z_06102020', 'filipeliu:narrative_1582914694010')

In [4]:
%run ../../cobrakbase/cobrakbase/core/kbasefba/newmodeltemplate.py
t2  = NewModelTemplate(copy.deepcopy(template_v2), 'tftr', 'tcpx')
t2z = NewModelTemplate(copy.deepcopy(template_v2z), 'tftr', 'tcpx')
#t3  = NewModelTemplate(copy.deepcopy(template_v3), 'tftr', 'tcpx')
#t3x = NewModelTemplate(copy.deepcopy(template_v3x), 'tftr', 'tcpx')
#t3z = NewModelTemplate(copy.deepcopy(template_v3z), 'tftr', 'tcpx')
#t3z = NewModelTemplate(copy.deepcopy(template_v3z), 'tftr', 'tcpx')


In [ ]:
o['formula'] = ""
for o in template.data['compounds']:
    if not type(o['mass']) == float:
        o['mass'] = float(o['mass'])
        print(o['id'], float(o['mass']))

In [ ]:
for o in template.data['compounds']:
    for k in o:
        if type(o[k]) == type(None):
            print(k)
        elif type(o[k]) == float and not o[k] == o[k]:
            print(o)
        elif o[k] == float('nan'):
            print(k)
for o in template.data['reactions']:
    for k in o:
        if type(o[k]) == type(None):
            print(k)
        elif o[k] == float('nan'):
            print(k)

In [104]:
o = t2z.data['complexes'][0]


{'complexroles': [{'optional_role': 0, 'templaterole_ref': '~/roles/id/ftr50048', 'triggering': 1}], 'confidence': 0, 'id': 'cpx50160', 'name': 'pscpx.160', 'reference': 'null', 'source': 'PlantSEED'}
ftr50048


In [7]:
from cobrakbase.core.kbasefba.newmodeltemplate_validator import NewModelTemplateValidator
print('t2z')
t2z_db = NewModelTemplateValidator(t2z)
t2z_db.validate_compounds()
t2z_db.validate()
print(len(t2z_db.undec_roles))
print(len(t2z_db.undec_compounds))

t2z
1131
0


In [ ]:
rxn07977_c0

In [68]:
tt = [t2_db, t3_db, t3x_db, t3z_db]
rxn_id = 'rxn07977_c'
rxn_id = 'rxn00851_c'
rxn_id = 'rxn09225_c'
for tdb in tt:
    if rxn_id in tdb.reactions:
        trxn = tdb.template.get_reaction(rxn_id)
        cpx_roles = trxn.get_complex_roles()
        for cpx_id in cpx_roles:

            for role_id in cpx_roles[cpx_id]:
                role = tdb.template.get_role(role_id)
                print(cpx_id, role_id, role['source'], role['name'])
        print(tdb.reactions[rxn_id]['templatecomplex_refs'])

cpx52598 ftr12940 ModelSEED Lipopolysaccharide core biosynthesis protein RfaS
['~/complexes/id/cpx00250', '~/complexes/id/cpx52598']
['~/complexes/id/cpx00250']
['~/complexes/id/cpx00250']
['~/complexes/id/cpx00250']


In [76]:
t2_db.template.get_complex('cpx52598')

{'complexroles': [{'optional_role': 0,
   'templaterole_ref': '~/roles/id/ftr12940',
   'triggering': 1}],
 'confidence': 1,
 'id': 'cpx52598',
 'name': 'Lipopolysaccharide core biosynthesis protein RfaS',
 'reference': 'Unknown',
 'source': 'SEED'}